# Test PyRotor

## Define setting

### Define quadratic cost function

In [1]:
# Quadratic part
q = np.array([1,0],
             [0,1])
# Linear part
w = np.array([0,0])

quad_model = [w, q]

### Define initial and final states

In [ ]:
endpoints = {'x1': {'start': .1,
                    'end': .9,
                    'delta': .01},
             'x2': {'start': .828,
                    'end': .172,
                    'delta': .01}}

### Define independent variable (time)

In [ ]:
independent_variable = {'start': .1,
                        'end': .9,
                        'frequency': .01}
# Compute number of evaluation points
delta_time = independent_variable['end'] - independent_variable['start']
delta_time /= independent_variable['frequency']
independent_variable['points_nb'] = int(delta_time) + 1

### Define constraints

In [ ]:
# Constraints
# x1 > 0
def f1(data):
    x1 = data["x1"].values
    return x1

# x1 < 1
def f2(data):
    x1 = data["x1"].values
    return 1 - x1

# x2 > 0
def f3(data):
    x2 = data["x2"].values
    return x2

# x2 < 1
def f4(data):
    x2 = data["x2"].values
    return 1 - x2

# x2 > f(x1)
def f5(data):
    x1 = data["x1"].values
    x2 = data["x2"].values
    return x2 - 150/19 * (1-x1)**3 + 225/19 * (1-x1)**2 - 100/19 * (1-x1) + 79/190

constraints = [f1, f2, f3, f4, f5]

### Define functional basis

In [ ]:
# Basis name
basis = 'legendre'
# Dimension for each variable
basis_dimension = {'x1': 5,
                   'x2': 5}

### Define iteration setting

In [ ]:
iteration_setting = {'early_stopping_threshold': }

### Import reference trajectories

## Optimization

### Create PyRotor class

### Execute PyRotor solver

### Compute savings

## Plot

In [ ]:
X = np.linspace(independent_variable['start'],
                independent_variable['end'],
                independent_variable['points_nb'])
X_ = np.linspace(0, 1, 101)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.plot(X, y_optimized['x1'])
ax3.set_xlabel('$t$')
ax3.set_ylabel('$x_1(t)$')
ax2.plot(X, y_optimized['x2'])
ax3.set_xlabel('$t$')
ax3.set_ylabel('$x_2(t)$')
ax3.plot(y_optimized['x1'], y_optimized['x2'])
ax3.fill_between(X_, 0, constraint_f5, color='r', alpha=.5, label='Forbidden area')
ax3.set_xlabel('$x_1$')
ax3.set_ylabel('$x_2$')
ax3.set_xlim(left=0, right=1)
ax3.set_ylim(bottom=0, top=1)
ax3.legend()
plt.tight_layout()